In [1]:
%matplotlib inline

In [2]:
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.dates as mdates
from matplotlib import rcParams

import numpy as np
import pandas as pd


import shapely.geometry as sgeom

import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader

import dask.dataframe as dd
import dask.bag as db
import dask.diagnostics as dg

import geopandas as gpd


paper_path = Path('../../paper/figures/')

slide_path = Path('../../slides/figures/intro/')
proposal_path = Path('../../draft/figures/intro/')
#rcParams['font.family'] = 'Segoe Print'

In [3]:
YEAR = 2021
names = ['ID', 'DATE', 'ELEMENT', 'DATA_VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG', 'OBS-TIME']
ds = dd.read_csv(f's3://noaa-ghcn-pds/csv/{YEAR}.csv', storage_options={'anon':True},  names=names, memory_map=False, 
                  dtype={'DATA_VALUE':'object'}, parse_dates=['DATE', 'OBS-TIME'])

In [4]:
# {column name:extents of the fixed-width fields}
columns = {"ID": (0,11), "LATITUDE": (12, 20), "LONGITUDE": (21, 30), "ELEVATION": (31, 37),"STATE": (38, 40),
           "NAME": (41, 71), "GSN FLAG": (72, 75), "HCN/CRN FLAG": (76, 79),"WMO ID": (80, 85)}

In [5]:
df = pd.read_fwf("http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-stations.txt", 
                    colspecs=list(columns.values()), names=list(columns.keys())).dropna(subset=['STATE'])

In [6]:
nydf = df[df['STATE'].str.match("NY")]

In [7]:
ds['ELEMENT'].unique().compute()

0     TMAX
1     TMIN
2     PRCP
3     TAVG
4     SNWD
      ... 
59    TSUN
60    DWPR
61    WT07
62    SN36
63    WT10
Name: ELEMENT, Length: 64, dtype: object

In [8]:
nyds = ds[ds['ID'].isin(nydf['ID'].unique()) & ds['ELEMENT'].isin(['TAVG', 'PRCP' ])].compute()

In [9]:
ny = nyds.merge(nydf, on='ID')

In [10]:
ny.columns

Index(['ID', 'DATE', 'ELEMENT', 'DATA_VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG',
       'OBS-TIME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'STATE', 'NAME',
       'GSN FLAG', 'HCN/CRN FLAG', 'WMO ID'],
      dtype='object')

In [ ]:
ds['ELEMENT'].unique().compute()

In [11]:
nyd = ny.pivot(index=['NAME','LATITUDE', 'LONGITUDE', 'DATE'], columns=['ELEMENT'], values=['DATA_VALUE']).reset_index()

In [13]:
nyd

NAME LATITUDE LONGITUDE       DATE DATA_VALUE     
ELEMENT                                                        PRCP TAVG
0           ALBANY 0.3 ESE  42.6641  -73.7935 2021-03-31          0  NaN
1           ALBANY 0.3 ESE  42.6641  -73.7935 2021-04-01         64  NaN
2           ALBANY 0.3 ESE  42.6641  -73.7935 2021-04-02         97  NaN
3           ALBANY 0.3 ESE  42.6641  -73.7935 2021-04-03          0  NaN
4           ALBANY 0.3 ESE  42.6641  -73.7935 2021-04-04          0  NaN
...                    ...      ...       ...        ...        ...  ...
43916    YOUNGSTOWN 2.2 NE  43.2671  -79.0068 2021-04-09         28  NaN
43917    YOUNGSTOWN 2.2 NE  43.2671  -79.0068 2021-04-10          0  NaN
43918    YOUNGSTOWN 2.2 NE  43.2671  -79.0068 2021-04-11         86  NaN
43919    YOUNGSTOWN 2.2 NE  43.2671  -79.0068 2021-04-12         41  NaN
43920    YOUNGSTOWN 2.2 NE  43.2671  -79.0068 2021-04-13         18  NaN

[43921 rows x 6 columns]

In [14]:
nyd.columns = [c[0] if i <4 else c[1] for i, c in enumerate(nyd.columns.to_flat_index())]

In [15]:
nyd.head()

,NAME,LATITUDE,LONGITUDE,DATE,PRCP,TAVG
0,ALBANY 0.3 ESE,42.6641,-73.7935,2021-03-31,0,NaN
1,ALBANY 0.3 ESE,42.6641,-73.7935,2021-04-01,64,NaN
2,ALBANY 0.3 ESE,42.6641,-73.7935,2021-04-02,97,NaN
3,ALBANY 0.3 ESE,42.6641,-73.7935,2021-04-03,0,NaN
4,ALBANY 0.3 ESE,42.6641,-73.7935,2021-04-04,0,NaN


In [16]:
nyd['TAVG'] =  (nyd['TAVG'].astype(float)/10) *(9/5) + 32

In [17]:
nyd.to_csv("nydata", index=False)

In [56]:
nyd['TAVG'] = 

SyntaxError: invalid syntax (<ipython-input-56-85f3640a2ebe>, line 1)

In [ ]:
ny.pivot?